In [1]:
import findspark
#findspark.init('/usr/hdp/2.6.1.0-129/spark2')
findspark.init()
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
from pyspark import SparkContext
sc=SparkContext()

In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
s_df = sqlContext.read.csv("big_data_log/big_data-2019-02-*.log", sep=';').cache()

In [5]:
s_products = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='products',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()

categories = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='categories',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load() 
          
products_categories = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='products_categories',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()
          
categories_content = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='categories_content',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()
          
products_content = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='products_content',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()

orders = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='orders',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()

order_items = sqlContext.read.format('jdbc').options(
          url='jdbc:mysql://10.1.3.244/tm_datasc',
          driver='com.mysql.jdbc.Driver',
          dbtable='order_items',
          user='datasc',
          password='scdata',
          serverTimezone='GMT',
          zeroDateTimeBehavior='CONVERT_TO_NULL').load()

In [7]:
s_df = s_df.selectExpr('_c0 as data_id', 
                   '_c1 as data_type', 
                   '_c2 as view_type',
                   '_c3 as customer_id',
                   '_c4 as vote_index',
                   '_c5 as added_qty',
                   '_c6 as token',
                   '_c7 as log_time',
                   '_c8 as session_id',
                   '_c9 as ip_address',
                   '_c10 as device_type',
                   '_c11 as click_success')

s_df = s_df.select(s_df.data_id.cast("integer"),
               s_df.view_type.cast("integer"),
               s_df.data_type.cast("integer"),
               s_df.customer_id.cast("integer"),
               s_df.added_qty.cast("integer"),
               s_df.token.cast("string"),
               s_df.log_time.cast("string"),
               s_df.session_id.cast("string"),
               s_df.ip_address.cast("string"),
               s_df.device_type.cast("integer"),
               s_df.click_success.cast("integer"))


s_df = s_df.fillna({'click_success':'0'})
s_df = s_df.fillna({'added_qty':'0'})
s_df = s_df.fillna({'data_id':'0'})
s_df = s_df.fillna({'view_type':'0'})
s_df = s_df.fillna({'data_type':'0'})

s_df = s_df.filter(s_df.click_success<3)
s_df = s_df.filter(s_df.added_qty<100)
s_df = s_df.filter(s_df.view_type<25)
s_df = s_df.filter(s_df.data_type<25)
s_df = s_df.filter(s_df.device_type<2)
s_df = s_df.filter(s_df.click_success<3)

In [8]:
from pyspark.sql.types import IntegerType, TimestampType
from pyspark.sql.functions import unix_timestamp, from_unixtime

s_df = s_df.withColumn('log_time', unix_timestamp("log_time", "yyyy-MM-dd HH:mm:ss").cast('timestamp').alias("log_time"))
s_df = s_df.withColumnRenamed('customer_cookie_token', 'token')
s_df = s_df.drop('id')
s_df = s_df.cache()

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import col, size
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import datediff, to_date, lit

break_point = '2017-04-29 00:00:00'

s_df = s_df.filter(s_df.log_time>break_point)
s_df = s_df.withColumn('date', s_df['log_time'].cast('date'))
s_df = s_df.withColumn('prod', F.when(s_df.data_type == 1, s_df.data_id).otherwise(0))
s_df = s_df.withColumn('cat', F.when(s_df.data_type == 7, s_df.data_id).otherwise(0))
s_df = s_df.withColumn('class', F.when(s_df.click_success == 1, s_df.click_success).otherwise(0))

In [10]:
s_df = s_df.dropDuplicates()

In [11]:
from pyspark.sql.functions import date_format

s_df = s_df.withColumn('year', year("log_time").alias('year'))
s_df = s_df.withColumn('month', month("log_time").alias('month'))
s_df = s_df.withColumn('wday', date_format('date', 'u'))
s_df = s_df.withColumn('hour', hour("log_time").alias('hour'))
s_df = s_df.withColumn('minute', minute("log_time").alias('minute'))
s_df = s_df.select(s_df.data_id,
               s_df.view_type,
               s_df.data_type,
               s_df.customer_id,
               s_df.added_qty,
               s_df.token,
               s_df.log_time,
               s_df.session_id,
               s_df.ip_address,
               s_df.device_type,
               s_df.click_success,
               s_df.date,
               s_df.prod.cast("integer"),
               s_df.cat.cast("integer"),
               s_df.year.cast("integer"),
               s_df.month.cast("integer"),
               s_df.wday.cast("integer"),
               s_df.hour.cast("integer"),
               s_df.minute.cast("integer"))
s_df = s_df.cache()

In [12]:
s_df = s_df.filter("token is not NULL")
s_df = s_df.filter("session_id is not NULL")
s_df = s_df.filter("log_time is not NULL")
s_df = s_df.filter("data_type is not NULL")
s_df = s_df.filter("view_type is not NULL")
s_df = s_df.filter("added_qty is not NULL")
s_df = s_df.filter("year is not NULL")
s_df = s_df.filter("month is not NULL")
s_df = s_df.filter("wday is not NULL")
s_df = s_df.filter("hour is not NULL")
s_df = s_df.filter("minute is not NULL")
s_df = s_df.filter("prod is not NULL")

In [13]:
categories = categories_content.select('category_id', 'name')
categories = categories.withColumnRenamed("name", "category_name")

products_content = products_content.select('product_id', 'name', 'type')
products_content = products_content.withColumnRenamed("name", "product_name")

s_products = s_products.select('product_id', 'brand', 'cena_niza_redovna', 'status')
s_products = s_products.withColumnRenamed("cena_niza_redovna", "price")
s_products = s_products.filter('status==1')

In [14]:
s_products = s_products.alias('a').join(products_content.alias('b'), s_products['product_id']==products_content['product_id'], 'inner').select('a.product_id', 'b.product_name', 'b.type', 'a.brand', 'a.price')

s_products = s_products.alias('a').join(products_categories.alias('b'), s_products['product_id']==products_categories['product_id'], 'inner').select('a.product_id', 'a.product_name', 'a.type', 'b.category_id', 'a.brand', 'a.price')

s_products = s_products.alias('a').join(categories.alias('b'), s_products['category_id']==categories['category_id'], 'inner').select('a.product_id', 'a.product_name', 'a.type', 'b.category_id', 'b.category_name','a.brand', 'a.price')

In [15]:
from pyspark.sql.functions import monotonically_increasing_id

tokens = s_df.select('token').distinct()
tokens = tokens.withColumn("user", monotonically_increasing_id())

tokens = tokens.cache()

s_df = s_df.alias('a').join(tokens.alias('b'), s_df['token'] == tokens['token'], 'left')\
.select('a.data_id', 'a.view_type', 'a.data_type', 'a.customer_id', 'a.added_qty', 'a.token', 'b.user', 'a.log_time', 'a.ip_address', 'a.session_id', 'a.device_type', 'a.click_success', 'a.prod', 'a.cat', 'a.date','a.year', 'a.month', 'a.wday', 'a.hour', 'a.minute')\
.cache()

In [16]:
s_df.count()

1670512

In [17]:
s_df.show()

+-------+---------+---------+-----------+---------+--------------------+----+-------------------+---------------+--------------------+-----------+-------------+------+---+----------+----+-----+----+----+------+
|data_id|view_type|data_type|customer_id|added_qty|               token|user|           log_time|     ip_address|          session_id|device_type|click_success|  prod|cat|      date|year|month|wday|hour|minute|
+-------+---------+---------+-----------+---------+--------------------+----+-------------------+---------------+--------------------+-----------+-------------+------+---+----------+----+-----+----+----+------+
|  75936|        7|        1|          0|        0|000344tlgncv5k5lr...|1059|2019-02-03 19:59:03|   66.249.92.33|000344tlgncv5k5lr...|          0|            0| 75936|  0|2019-02-03|2019|    2|   7|  19|    59|
| 128207|        7|        1|          0|        0|00750610f0d326cb6...| 235|2019-02-08 18:07:20|  93.87.106.241|pvt567f3l2oaadded...|          0|          

In [18]:
s_df.printSchema()

root
 |-- data_id: integer (nullable = true)
 |-- view_type: integer (nullable = true)
 |-- data_type: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- added_qty: integer (nullable = true)
 |-- token: string (nullable = true)
 |-- user: long (nullable = true)
 |-- log_time: timestamp (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- click_success: integer (nullable = true)
 |-- prod: integer (nullable = true)
 |-- cat: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- wday: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)



In [19]:
s_df = s_df.filter("customer_id>0")
s_df.count()

7624

In [20]:
s_df = s_df.withColumn("current_date", F.date_format(F.current_date(), "y-M-d")).withColumn("log_date", F.date_format(s_df.date, "y-M-d"))
s_df = s_df.withColumn('current_date', s_df['current_date'].cast('date'))
s_df = s_df.withColumn('log_date', s_df['log_date'].cast('date'))
s_df = s_df.withColumn("days_between", F.datediff(s_df.current_date, s_df.date))

In [21]:
s_df.printSchema()

root
 |-- data_id: integer (nullable = true)
 |-- view_type: integer (nullable = true)
 |-- data_type: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- added_qty: integer (nullable = true)
 |-- token: string (nullable = true)
 |-- user: long (nullable = true)
 |-- log_time: timestamp (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- click_success: integer (nullable = true)
 |-- prod: integer (nullable = true)
 |-- cat: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- wday: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- current_date: date (nullable = true)
 |-- log_date: date (nullable = true)
 |-- days_between: integer (nullable = true)



In [22]:
s_df.select("customer_id", "log_date", "days_between").show()

+-----------+----------+------------+
|customer_id|  log_date|days_between|
+-----------+----------+------------+
|      32477|2019-02-04|           8|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     544369|2019-02-02|          10|
|     208691|2019-02-10|           2|
|     208691|2019-02-10|           2|
|     156367|2019-02-05|           7|
|     444110|2019-02-06|           6|
|     444110|2019-02-06|           6|
|     444110|2019-02-06|           6|
|     444110|2019-02-06|           6|
|     528559|2019-02-06|           6|
|     528559|2019-02-06|           6|
|     528559|2019-02-06|           6|
|     528559|2019-02-06|           6|
|     528559|2019-02-06|           6|
+-----------+----------+------------+
only showing top 20 rows



In [23]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(s_df['customer_id']).orderBy(s_df['log_date'].asc())

In [24]:
n=1
first_transaction = s_df.select(col('customer_id'), col('log_date'), row_number().over(window).alias('row_number')) \
  .where(col('row_number') <= n).drop('row_number')

In [25]:
first_transaction = first_transaction.withColumnRenamed('log_date', 'first_transaction')

In [26]:
window = Window.partitionBy(s_df['customer_id']).orderBy(s_df['log_date'].desc())

In [27]:
last_transaction = s_df.select(col('customer_id'), col('log_date'), row_number().over(window).alias('row_number')) \
  .where(col('row_number') <= n).drop('row_number')

In [28]:
last_transaction = last_transaction.withColumnRenamed('log_date', 'last_transaction')

In [29]:
#shopping_frequency (num of orders per customer_id)
frequency = s_df.groupBy("customer_id", "data_type").count().filter("data_type==11").orderBy("count", ascending=False).drop('data_type')

In [30]:
#recency (difference between last transaction and today)
recency = last_transaction.withColumn("current_date", F.date_format(F.current_date(), "y-M-d")).withColumn("log_date", F.date_format(last_transaction.last_transaction, "y-M-d"))
recency = recency.withColumn("days_between", F.datediff(recency.current_date, recency.last_transaction))
recency = recency.withColumnRenamed('days_between','recency').drop('last_transaction', 'current_date')

In [31]:
length = first_transaction.join(last_transaction, ['customer_id'], 'inner')

In [32]:
length = length.withColumn("length", F.datediff(length.last_transaction, length.first_transaction))

In [33]:
length.show(100)

+-----------+-----------------+----------------+------+
|customer_id|first_transaction|last_transaction|length|
+-----------+-----------------+----------------+------+
|      94819|       2019-02-02|      2019-02-02|     0|
|     413073|       2019-01-31|      2019-01-31|     0|
|     459999|       2019-02-07|      2019-02-07|     0|
|     493490|       2019-02-07|      2019-02-07|     0|
|     547213|       2019-02-08|      2019-02-09|     1|
|     105437|       2019-01-31|      2019-02-08|     8|
|     444553|       2019-01-31|      2019-01-31|     0|
|     544177|       2019-01-31|      2019-01-31|     0|
|     545725|       2019-02-05|      2019-02-05|     0|
|     545792|       2019-02-05|      2019-02-05|     0|
|     546883|       2019-02-07|      2019-02-07|     0|
|     502742|       2019-02-01|      2019-02-07|     6|
|     544777|       2019-02-01|      2019-02-02|     1|
|     546284|       2019-02-06|      2019-02-06|     0|
|     546829|       2019-02-07|      2019-02-08|

In [34]:
length.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- first_transaction: date (nullable = true)
 |-- last_transaction: date (nullable = true)
 |-- length: integer (nullable = true)



In [35]:
recency.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- log_date: string (nullable = true)
 |-- recency: integer (nullable = true)



In [36]:
#fali monetary i frequency

In [37]:
s_df.printSchema()

root
 |-- data_id: integer (nullable = true)
 |-- view_type: integer (nullable = true)
 |-- data_type: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- added_qty: integer (nullable = true)
 |-- token: string (nullable = true)
 |-- user: long (nullable = true)
 |-- log_time: timestamp (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- click_success: integer (nullable = true)
 |-- prod: integer (nullable = true)
 |-- cat: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- wday: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- current_date: date (nullable = true)
 |-- log_date: date (nullable = true)
 |-- days_between: integer (nullable = true)



In [38]:
frequency = s_df.groupBy("customer_id").count().withColumnRenamed("count", "frequency")

In [39]:
frequency.show()

+-----------+---------+
|customer_id|frequency|
+-----------+---------+
|     547213|       19|
|     493490|        3|
|     459999|        3|
|      94819|        2|
|     413073|        3|
|     544177|        4|
|     444553|        2|
|     545725|        6|
|     105437|        6|
|     546883|        8|
|     545792|       14|
|     546829|        4|
|     544777|       47|
|     502742|        3|
|     547827|        5|
|     546284|        2|
|     545983|        2|
|     545186|        8|
|     543014|        1|
|     544231|        3|
+-----------+---------+
only showing top 20 rows



In [40]:
monetary = s_df.filter("data_type==11")

In [41]:
orders = orders.select("order_id", "customer_id", "total_value")

In [42]:
monetary = monetary.join(orders, monetary['data_id']==orders['order_id'], 'inner').select(monetary['customer_id'], 'total_value')

In [43]:
monetary = monetary.groupBy("customer_id").sum()

In [44]:
monetary = monetary.drop("sum(customer_id)").withColumnRenamed("sum(total_value)", "monetary")

In [45]:
monetary.show()

+-----------+--------+
|customer_id|monetary|
+-----------+--------+
|     413073|   29479|
|     459999|   29499|
|      94819|   33051|
|     444553|    2580|
|     545792|   34290|
|     105437|   21591|
|     546284|    5990|
|     545983|    7999|
|     547143|    1788|
|     544035|    5390|
|     547683|   47898|
|     544231|    3861|
|     547645|   37399|
|     519133|   49090|
|     545546|    3699|
|     544347|   35991|
|     541850|   65980|
|     546758|   24439|
|     545024|   25990|
|     546969|    2029|
+-----------+--------+
only showing top 20 rows



In [46]:
print("Monetary: ",monetary.count())
print("Frequency: ",frequency.count())
print("Length: ",length.count())
print("Recency: ",recency.count())

Monetary:  614
Frequency:  1224
Length:  1224
Recency:  1224


In [47]:
lrfm = frequency.join(length, ['customer_id'], 'inner').join(recency, ['customer_id'], 'inner').join(monetary, ['customer_id'], 'left').fillna(0)

In [48]:
lrfm = lrfm.drop("first_transaction", "last_transaction", "log_date")

In [49]:
lrfm.show()

+-----------+---------+------+-------+--------+
|customer_id|frequency|length|recency|monetary|
+-----------+---------+------+-------+--------+
|      94819|        2|     0|     10|   33051|
|     413073|        3|     0|     12|   29479|
|     459999|        3|     0|      5|   29499|
|     493490|        3|     0|      5|       0|
|     547213|       19|     1|      3|       0|
|     105437|        6|     8|      4|   21591|
|     444553|        2|     0|     12|    2580|
|     544177|        4|     0|     12|       0|
|     545725|        6|     0|      7|       0|
|     545792|       14|     0|      7|   34290|
|     546883|        8|     0|      5|       0|
|     502742|        3|     6|      5|       0|
|     544777|       47|     1|     10|       0|
|     546284|        2|     0|      6|    5990|
|     546829|        4|     1|      4|       0|
|     547827|        5|     0|      2|       0|
|      18884|       24|     2|      2|       0|
|     439105|        2|     0|      3|  

In [51]:
pseudo_products = sqlContext.read.csv("cbr.csv", sep=',', header = True).cache()

In [59]:
customer_category = s_df.join(s_products, s_df['prod']==s_products['product_id'], 'inner').select('customer_id', 'product_id', 'category_name')

In [60]:
customer_category.show()

+-----------+----------+--------------------+
|customer_id|product_id|       category_name|
+-----------+----------+--------------------+
|     545151|     40634|Samostalne mikrot...|
|     545151|     40634|Samostalne mikrot...|
|     545151|     40634|Samostalne mikrot...|
|     545151|     40634|Samostalne mikrot...|
|     542642|     40634|Samostalne mikrot...|
|     542642|     40634|Samostalne mikrot...|
|     542642|     40634|Samostalne mikrot...|
|     542642|     40634|Samostalne mikrot...|
|     542642|     40634|Samostalne mikrot...|
|     273903|     49586|       Salamoreznice|
|     235193|     58289|             Tiganji|
|     191841|     62895|Wi-Fi, Bluetooth ...|
|     235193|     63710|      Inverter klime|
|     536425|     63710|      Inverter klime|
|     536425|     63710|      Inverter klime|
|     327457|     67148|       Ugradne ploče|
|     462351|     67148|       Ugradne ploče|
|     462351|     67148|       Ugradne ploče|
|     544777|     74723|      Inve

In [61]:
from pyspark.sql import functions as F

grouped = customer_category.groupBy("customer_id", "category_name").count()
pivoted = grouped.groupBy("customer_id").pivot("category_name").agg(F.sum(F.col("count")))
pivoted = pivoted.na.fill(0)

In [65]:
pivoted = pivoted.join(lrfm, ['customer_id'], 'inner')

In [66]:
pivoted.show(5)

+-----------+----------------------------+--------------+------------------------+-------------------+-------------------+--------------------------+----------------------------+---------------------------+-----------------------------+-------------------+-------------------+----------------+------------------------------+-----------+--------------------------------+----------+-------------+----------------+----------------+----------------+-------+-----------------------------+---------------+---------------+--------------------------+-------------------+---+--------------------+----------------+--------------------+-----------------------+----------+----------------+------------------+---------------------------------+--------------------------+--------------------------+-------------------------+---------------------------+-------+--------------------------------+---------------------+-----------------+-------------------+--------------------------+------------------+---------+-----

In [68]:
pivoted.columns

['customer_id',
 'Adapteri, baterije i punjači',
 'Alati za baštu',
 'Antialergijski usisivači',
 'Aparati za brijanje',
 'Aparati za espresso',
 'Aparati za kuvanje na pari',
 'Aparati za mafine i cupcakes',
 'Aparati za negu lica i tela',
 'Aparati za pravljenje jogurta',
 'Aparati za pritisak',
 'Aparati za sladoled',
 'Aparati za vafle',
 'Aparati za vertikalno peglanje',
 'Auto kamere',
 'Auto punjači za mobilne telefone',
 'Auto radio',
 'Auto zvučnici',
 'Baterijske lampe',
 'Bežični telefoni',
 'Bežični zvučnici',
 'Bicikli',
 'Blenderi sa funkcijom kuvanja',
 'Blu-ray plejeri',
 'Blu-ray sistemi',
 'Bokali za filtriranje vode',
 'Bušilice i odvrtači',
 'CD',
 'Cediljke i citrusete',
 'DSLR fotoaparati',
 'Daljinski upravljači',
 'Daske i oprema za pegle',
 'Depilatori',
 'Desktop računari',
 'Dodaci za blendere',
 'Dodatna oprema za aparate za kafu',
 'Dodatna oprema za računare',
 'Dodatna oprema za sudopere',
 'Dodatna oprema za tablete',
 'Dodatna oprema za usisivače',
 'Dr

In [70]:
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import StandardScaler

In [71]:
vecAssembler = VectorAssembler(inputCols=['Adapteri, baterije i punjači',
 'Alati za baštu',
 'Antialergijski usisivači',
 'Aparati za brijanje',
 'Aparati za espresso',
 'Aparati za kuvanje na pari',
 'Aparati za mafine i cupcakes',
 'Aparati za negu lica i tela',
 'Aparati za pravljenje jogurta',
 'Aparati za pritisak',
 'Aparati za sladoled',
 'Aparati za vafle',
 'Aparati za vertikalno peglanje',
 'Auto kamere',
 'Auto punjači za mobilne telefone',
 'Auto radio',
 'Auto zvučnici',
 'Baterijske lampe',
 'Bežični telefoni',
 'Bežični zvučnici',
 'Bicikli',
 'Blenderi sa funkcijom kuvanja',
 'Blu-ray plejeri',
 'Blu-ray sistemi',
 'Bokali za filtriranje vode',
 'Bušilice i odvrtači',
 'CD',
 'Cediljke i citrusete',
 'DSLR fotoaparati',
 'Daljinski upravljači',
 'Daske i oprema za pegle',
 'Depilatori',
 'Desktop računari',
 'Dodaci za blendere',
 'Dodatna oprema za aparate za kafu',
 'Dodatna oprema za računare',
 'Dodatna oprema za sudopere',
 'Dodatna oprema za tablete',
 'Dodatna oprema za usisivače',
 'Dronovi',
 'Držači mobilnih telefona za auto',
 'Eksterni hard diskovi',
 'Električne džezve',
 'Električne vodenice',
 'Električne četkice za zube',
 'Električni šporeti',
 'Epilatori',
 'Escajg',
 'FM transmiteri',
 'Fenovi za kosu',
 'Filteri za bokale za vodu',
 'Fitbit',
 'Fitnes oprema',
 'Friteze',
 'Frižideri sa jednim vratima',
 'Grafičke karte',
 'Gramofoni',
 'Grejalice',
 'Horizontalni zamrzivači',
 'Indukcijski rešoi',
 'Indukcijski šporeti',
 'Inverter klime',
 'Jastuci',
 'Jonizatori',
 'Kablovi za TV / AV',
 'Kablovi, adapteri i baterije',
 'Kafa',
 'Kalkulatori',
 'Kamini i peći na čvrsta goriva',
 'Kertridži i toneri',
 'Kese za vakumiranje',
 'Kombinovani frižideri',
 'Kombinovani šporeti',
 'Konzole',
 'Kuhinjske vage',
 'Kuhinjski pribor',
 'Kuhinjski roboti',
 'Kuvala za vodu',
 'Laptop računari',
 'MP3 plejeri',
 'Masažeri',
 'Matične ploče',
 'Mašine za meso',
 'Mašine za pranje i sušenje veša',
 'Mašine za pranje veša',
 'Mašine za sušenje veša',
 'Memorijske kartice',
 'Meteorološke stanice',
 'Mikrofoni',
 'Mikseri sa posudom',
 'Mini barovi',
 'Mini kuhinje',
 'Mini linije',
 'Mini šporeti',
 'Miševi',
 'Monitori',
 'Multi trimeri',
 'Multicookeri',
 'Multipraktici',
 'Mutilice za nes kafu',
 'Napajanja',
 'Nosači i police',
 'Nutri blenderi',
 'Odvlaživači',
 'Oprema za aparate za brijanje',
 'Oprema za frižidere',
 'Oprema za igrice',
 'Oprema za mašine za veš',
 'Oprema za mikrotalasne rerne',
 'Oprema za sudomašine',
 'Oprema za šivaće mašine',
 'Oprema za šporete i rerne',
 'Ostala oprema za kuću',
 'Ostala oprema za telefone',
 'Ovlaživači',
 'Oštrači noževa',
 'PCI kartice',
 'Pametni satovi',
 'Parne stanice',
 'Paročistači',
 'Pekare',
 'Pekači i tepsije',
 'Plastične kutije za hranu',
 'Plinski šporeti',
 'Podloge za miš',
 'Pokretne klime',
 'Postolja za laptopove',
 'Power Bank',
 'Prese za kosu',
 'Procesori',
 'Programi i softveri',
 'Punjači i baterije',
 'Punjači za mobilne telefone',
 'Radijatori',
 'Rashladne vinske vitrine',
 'Robotski usisivači',
 'Roštilji i grilovi',
 'Ruteri',
 'Ručni mikseri',
 'Ručni usisivači',
 'Salamoreznice',
 'Samostalne mašine za pranje sudova',
 'Samostalne mikrotalasne rerne',
 'Samostalni aspiratori',
 'Set-top box tjuneri i risiveri',
 'Setovi posuđa',
 'Setovi ručnog alata',
 'Side by side frižideri',
 'Sijalice',
 'Slušalice',
 'Smart telefoni',
 'Smoothie blenderi',
 'Sobne antene',
 'Soundbar zvučnici',
 'Standardne klime',
 'Standardne pegle',
 'Standardne seckalice',
 'Standardni blenderi',
 'Standardni bojleri',
 'Standardni rešoi',
 'Standardni sokovnici',
 'Standardni telefoni',
 'Standardni tosteri',
 'Stone sudomašine',
 'Styleri',
 'Sudopere',
 'Sušilice za veš',
 'Tableti',
 'Tastature',
 'Televizori',
 'Tiganji',
 'Toplomeri',
 'Torbe i futrole',
 'Torbice i futrole za mobilne telefone',
 'Tosteri za sendviče',
 'Tranzistori i radio uređaji',
 'Trimeri za bradu',
 'Trimeri za kosu',
 'Trimeri za telo',
 'USB flashevi',
 'Ugradne mašine za sudove',
 'Ugradne mikrotalasne rerne',
 'Ugradne ploče',
 'Ugradne rerne',
 'Ugradni aspiratori',
 'Ugradni frižideri',
 'Ugradni setovi',
 'Usisivači sa kesom za prašinu',
 'Usisivači sa posudom za prašinu',
 'Usisivači sa vodenim filterom',
 'Vage za merenje telesne težine',
 'Vakum aparati',
 'Vertikalni zamrzivači',
 'Wi-Fi, Bluetooth i USB adapteri',
 'Zvučnici za kompjuter',
 'iPad',
 'Čitači kartica',
 'Šerpe i lonci',
 'Šivaće mašine',
 'Šporeti na čvrsto gorivo',
 'Štampači',
 'Štapni mikseri',
 'Štapni usisivači',
 'Žični telefoni',
 'frequency',
 'length',
 'recency',
 'monetary'], outputCol='features')

In [72]:
new_df = vecAssembler.transform(pivoted)

scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')
scaler_model = scaler.fit(new_df)
scaled_data = scaler_model.transform(new_df)
scaled_data.select('features','scaledFeatures').show(truncate=False)

+-------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                               |scaledFeatures                                                                                                                                                                                                                                                                    |
+-------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------

In [102]:
from pyspark.ml.clustering import BisectingKMeans

# Trains a bisecting k-means model.
bkm = BisectingKMeans().setK(10).setSeed(1)
model = bkm.fit(scaled_data)

transformed = model.transform(scaled_data)

In [107]:
scaled_data.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- Adapteri, baterije i punjači: long (nullable = true)
 |-- Alati za baštu: long (nullable = true)
 |-- Antialergijski usisivači: long (nullable = true)
 |-- Aparati za brijanje: long (nullable = true)
 |-- Aparati za espresso: long (nullable = true)
 |-- Aparati za kuvanje na pari: long (nullable = true)
 |-- Aparati za mafine i cupcakes: long (nullable = true)
 |-- Aparati za negu lica i tela: long (nullable = true)
 |-- Aparati za pravljenje jogurta: long (nullable = true)
 |-- Aparati za pritisak: long (nullable = true)
 |-- Aparati za sladoled: long (nullable = true)
 |-- Aparati za vafle: long (nullable = true)
 |-- Aparati za vertikalno peglanje: long (nullable = true)
 |-- Auto kamere: long (nullable = true)
 |-- Auto punjači za mobilne telefone: long (nullable = true)
 |-- Auto radio: long (nullable = true)
 |-- Auto zvučnici: long (nullable = true)
 |-- Baterijske lampe: long (nullable = true)
 |-- Bežični telefoni: long (nu

In [108]:
#Evaluate clustering
cost = model.computeCost(scaled_data)
print("Within Set Sum of Squared Errors = " + str(cost))

Within Set Sum of Squared Errors = 10533570386.089205


In [103]:
transformed.groupBy('prediction').count().sort(col('prediction').asc()).show(15)

+----------+-----+
|prediction|count|
+----------+-----+
|         0|  578|
|         1|  105|
|         2|   89|
|         3|   43|
|         4|   50|
|         5|   35|
|         6|   10|
|         7|    7|
|         8|    2|
|         9|    2|
+----------+-----+



In [90]:
grouped.filter("customer_id=='546817'").show(truncate=False)

+-----------+----------------------+-----+
|customer_id|category_name         |count|
+-----------+----------------------+-----+
|546817     |Mašine za sušenje veša|4    |
+-----------+----------------------+-----+



In [86]:
transformed.select("customer_id", "prediction").filter("prediction == 4").show()

+-----------+----------+
|customer_id|prediction|
+-----------+----------+
|     459999|         4|
|     545792|         4|
|     529730|         4|
|     237621|         4|
|     355766|         4|
|     545925|         4|
|     545937|         4|
|     252427|         4|
|     544461|         4|
|     544441|         4|
|     440798|         4|
|     544004|         4|
|     544153|         4|
|     547861|         4|
|     544885|         4|
|     547328|         4|
|     492829|         4|
|     547365|         4|
|     545253|         4|
|     546817|         4|
+-----------+----------+
only showing top 20 rows



In [93]:
s_products.select("category_name").distinct().count()

334